In [ ]:
!pip install beautifulsoup4

In [ ]:
from bs4 import BeautifulSoup
import requests as r
import time
import pandas as pd

In [ ]:
def scrap_wikipedia_keyword(keyword:str)->str:
    str_keyword_html=keyword.replace(" ","%20")
    req=r.get(f"https://fr.wikipedia.org/wiki/{str_keyword_html}")
    soup = BeautifulSoup(req.text)
    list_phrases = [element.text for element in soup.find_all("p") if keyword.lower() in element.text.lower()]
    if "homonymie" in req.text:
        print("homonymie")
    if list_phrases :
        return  list_phrases[0]
    return ""

In [ ]:
with open("../data/ingredients.csv","r") as f:
    list_ingredients=f.readlines()[1:]
    list_ingredients=list(map(lambda x: x.replace("\n","").lower(),list_ingredients))

In [ ]:
list_phrase_exemple=list()
df = pd.DataFrame(columns=["keyword", "sentence", "indexes"],dtype=object)

for ingredient in  list_ingredients:
    print(f"process {ingredient}")
    phrase=scrap_wikipedia_keyword(ingredient)
    start=phrase.find(ingredient)
    try:
        new_line = {
            "keyword": ingredient,
            "sentence": phrase,
            "indexes": (start,start+len(ingredient)),
        }
    except:
        print(f"bug {ingredient}")
            
        
    df_new_row = pd.DataFrame.from_records(data=new_line)
    df = pd.concat([df, df_new_row])
    print(f"finish {ingredient} in {phrase}")
    time.sleep(1)
    

In [ ]:
df2=df[["keyword","sentence"]]
df2=df2.drop_duplicates()

In [ ]:
df2

In [ ]:
df2["indexes"]=df2[["keyword","sentence"]].apply(lambda x : x["sentence"].lower().find(x["keyword"]),axis=1)
df2["indexes2"]=df2[["keyword","indexes"]].apply(lambda x : x["indexes"]+len(x["keyword"]),axis=1)

In [ ]:
df2.to_parquet("train_data.parquet")

In [ ]:
#training_data = [
#  ("Tokyo Tower is 333m tall.", [(0, 11, "BUILDING")]),
#]


In [ ]:
with open("sentence.txt","w") as file_to_write:
    file_to_write.write("".join(list(df2["sentence"])))